<center>
<img src="https://github.com/oussou-dev/mlcourse.ai/blob/jupyter_french/img/ods_stickers.jpg?raw=true" />
<br>
    
<div style="font-weight:700;font-size:25px"> [mlcourse.ai](https://mlcourse.ai) - Open Machine Learning Course </div>

<br>
Auteur: [Yury Kashnitsky](https://yorko.github.io). Traduit et édité par [Christina Butsko](https://www.linkedin.com/in/christinabutsko/), [Nerses Bagiyan](https://www.linkedin.com/in/nersesbagiyan/), [Yulia Klimushina](https://www.linkedin.com/in/yuliya-klimushina-7168a9139), [Yuanyuan Pao](https://www.linkedin.com/in/yuanyuanpao/) et [Ousmane Cissé](https://github.com/oussou-dev). Ce matériel est soumis aux termes et conditions de la licence [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). L'utilisation gratuite est autorisée à des fins non commerciales.

# <center>Topic 4. Classification et régression linéaires
## <center> Partie 4. Où la régression logistique est bonne et où elle ne l'est pas

### Analyse des critiques de films IMDB

Maintenant pour un peu de pratique! Nous voulons résoudre le problème de la classification binaire des critiques de films IMDB. Nous avons un training set avec des critiques marquées, 12500 critiques marquées comme bonnes, 12500 comme mauvaises. Ici, il n’est pas facile de se lancer immédiatement dans l’apprentissage automatique car nous n’avons pas la matrice $X$; nous devons la préparer. Nous allons utiliser une approche simple: modèle de "sac de mots" (bag of words model). Les caractéristiques de la critique seront représentées par des indicateurs de la présence de chaque mot de l'ensemble du corpus dans cette critique. Le corpus est l'ensemble de toutes les critiques d'utilisateurs. L'idée est illustrée par une photo

<img src="https://github.com/oussou-dev/mlcourse.ai/blob/jupyter_french/img/bag_of_words.png?raw=true" width=80% />

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

**Pour commencer, nous téléchargeons automatiquement le jeu de données depuis [ici](http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz) et le désarchivons avec le reste des jeux de données. dans le dossier de données. L'ensemble de données est brièvement décrit [ici](http://ai.stanford.edu/~amaas/data/sentiment/). Il y a 12,5k de bonnes et de mauvaises critiques dans les données de test et de training set.**

In [ ]:
import tarfile
from io import BytesIO

import requests

url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"


def load_imdb_dataset(extract_path="../../data", overwrite=False):
    # check if existed already
    if (
        os.path.isfile(os.path.join(extract_path, "aclImdb", "README"))
        and not overwrite
    ):
        print("IMDB dataset is already in place.")
        return

    print("Downloading the dataset from:  ", url)
    response = requests.get(url)

    tar = tarfile.open(mode="r:gz", fileobj=BytesIO(response.content))

    data = tar.extractall(extract_path)


load_imdb_dataset()

In [ ]:
# change if you have it in alternative location
PATH_TO_IMDB = "../../data/aclImdb"

reviews_train = load_files(
    os.path.join(PATH_TO_IMDB, "train"), categories=["pos", "neg"]
)
text_train, y_train = reviews_train.data, reviews_train.target

reviews_test = load_files(os.path.join(PATH_TO_IMDB, "test"), categories=["pos", "neg"])
text_test, y_test = reviews_test.data, reviews_test.target

In [ ]:
# # Alternatively, load data from previously pickled objects.
# import pickle
# with open('../../data/imdb_text_train.pkl', 'rb') as f:
#     text_train = pickle.load(f)
# with open('../../data/imdb_text_test.pkl', 'rb') as f:
#     text_test = pickle.load(f)
# with open('../../data/imdb_target_train.pkl', 'rb') as f:
#     y_train = pickle.load(f)
# with open('../../data/imdb_target_test.pkl', 'rb') as f:
#     y_test = pickle.load(f)

In [ ]:
print("Number of documents in training data: %d" % len(text_train))
print(np.bincount(y_train))
print("Number of documents in test data: %d" % len(text_test))
print(np.bincount(y_test))

**Voici quelques exemples d’avis.**

In [ ]:
print(text_train[1])

In [ ]:
y_train[1]  # bad review

In [ ]:
text_train[2]

In [ ]:
y_train[2]  # good review

In [ ]:
# import pickle
# with open('../../data/imdb_text_train.pkl', 'wb') as f:
#     pickle.dump(text_train, f)
# with open('../../data/imdb_text_test.pkl', 'wb') as f:
#     pickle.dump(text_test, f)
# with open('../../data/imdb_target_train.pkl', 'wb') as f:
#     pickle.dump(y_train, f)
# with open('../../data/imdb_target_test.pkl', 'wb') as f:
#     pickle.dump(y_test, f)

## Un simple décompte de mots

**Tout d'abord, nous allons créer un dictionnaire de tous les mots en utilisant CountVectorizer**

In [ ]:
cv = CountVectorizer()
cv.fit(text_train)

len(cv.vocabulary_)

**Si vous regardez les exemples de "mots" (appelons-les jetons), vous constaterez que nous avons omis bon nombre des étapes importantes du traitement de texte (le traitement de texte automatique peut lui-même constituer une série d'articles complètement distinct). .**

In [ ]:
print(cv.get_feature_names()[:50])
print(cv.get_feature_names()[50000:50050])

**Deuxièmement, nous encodons les phrases des textes du training set avec les index des mots entrants. Nous utiliserons le format "clairsemé".**

In [ ]:
X_train = cv.transform(text_train)
X_train

**Voyons comment notre transformation a fonctionné**

In [ ]:
print(text_train[19726])

In [ ]:
X_train[19726].nonzero()[1]

In [ ]:
X_train[19726].nonzero()

**Troisièmement, nous appliquerons les mêmes opérations au jeu de test**

In [ ]:
X_test = cv.transform(text_test)

**La prochaine étape consiste à entraîner la régression logistique.**

In [ ]:
%%time
logit = LogisticRegression(solver="lbfgs", n_jobs=-1, random_state=7)
logit.fit(X_train, y_train)

**Let's look at accuracy on the both the training and the test sets.**

In [ ]:
round(logit.score(X_train, y_train), 3), round(logit.score(X_test, y_test), 3),

**Les coefficients du modèle peuvent être magnifiquement affichés.**

In [ ]:
def visualize_coefficients(classifier, feature_names, n_top_features=25):
    # get coefficients with large absolute values
    coef = classifier.coef_.ravel()
    positive_coefficients = np.argsort(coef)[-n_top_features:]
    negative_coefficients = np.argsort(coef)[:n_top_features]
    interesting_coefficients = np.hstack([negative_coefficients, positive_coefficients])
    # plot them
    plt.figure(figsize=(15, 5))
    colors = ["red" if c < 0 else "blue" for c in coef[interesting_coefficients]]
    plt.bar(np.arange(2 * n_top_features), coef[interesting_coefficients], color=colors)
    feature_names = np.array(feature_names)
    plt.xticks(
        np.arange(1, 1 + 2 * n_top_features),
        feature_names[interesting_coefficients],
        rotation=60,
        ha="right",
    );

In [ ]:
def plot_grid_scores(grid, param_name):
    plt.plot(
        grid.param_grid[param_name],
        grid.cv_results_["mean_train_score"],
        color="green",
        label="train",
    )
    plt.plot(
        grid.param_grid[param_name],
        grid.cv_results_["mean_test_score"],
        color="red",
        label="test",
    )
    plt.legend();

In [ ]:
visualize_coefficients(logit, cv.get_feature_names())

**Pour améliorer notre modèle, nous pouvons optimiser le coefficient de régularisation pour la «régression logistique». Nous utiliserons `sklearn.pipeline` car` CountVectorizer` ne devrait être appliqué qu'aux données d'apprentissage (afin de ne pas "jeter un coup d'œil" dans le jeu de test et de ne pas y compter la fréquence des mots). Dans ce cas, `pipeline` détermine la séquence d'actions appropriée: applique` CountVectorizer`, puis entraîne `Régression logistique`.**

In [ ]:
%%time
from sklearn.pipeline import make_pipeline

text_pipe_logit = make_pipeline(
    CountVectorizer(),
    # for some reason n_jobs > 1 won't work
    # with GridSearchCV's n_jobs > 1
    LogisticRegression(solver="lbfgs", n_jobs=1, random_state=7),
)

text_pipe_logit.fit(text_train, y_train)
print(text_pipe_logit.score(text_test, y_test))

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

param_grid_logit = {"logisticregression__C": np.logspace(-5, 0, 6)}
grid_logit = GridSearchCV(
    text_pipe_logit, param_grid_logit, return_train_score=True, cv=3, n_jobs=-1
)

grid_logit.fit(text_train, y_train)

**Imprimons les meilleurs $C$ et cv-score en utilisant cet hyperparamètre:**

In [ ]:
grid_logit.best_params_, grid_logit.best_score_

In [ ]:
plot_grid_scores(grid_logit, "logisticregression__C")

For the validation set:

In [ ]:
grid_logit.score(text_test, y_test)

**Maintenant, faisons la même chose avec la forêt aléatoire. Nous voyons qu'avec la régression logistique, nous obtenons une meilleure précision avec moins d'effort.**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=17)

In [ ]:
%%time
forest.fit(X_train, y_train)

In [ ]:
round(forest.score(X_test, y_test), 3)

### XOR-Problem
Considérons maintenant un exemple où les modèles linéaires sont pires.

Les méthodes de classification linéaire définissent encore une surface de séparation très simple - un hyperplan. L'exemple de jouet le plus célèbre où les classes ne peuvent pas être divisées par un hyperplan (ou une ligne) sans erreur est "le problème XOR".

XOR est le "OU exclusif", une fonction booléenne avec la table de vérité suivante:



<img src='https://github.com/oussou-dev/mlcourse.ai/blob/jupyter_french/img/XOR_table.gif?raw=true'>

XOR est le nom donné à un simple problème de classification binaire dans lequel les classes sont présentées sous forme de nuages ​​de points intersectants étendus en diagonale.

In [ ]:
# creating dataset
rng = np.random.RandomState(0)
X = rng.randn(200, 2)
y = np.logical_xor(X[:, 0] > 0, X[:, 1] > 0)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], s=30, c=y, cmap=plt.cm.Paired);

De toute évidence, on ne peut pas tracer une seule ligne droite pour séparer une classe d'une autre sans erreurs. Par conséquent, la régression logistique fonctionne mal avec cette tâche.

In [ ]:
def plot_boundary(clf, X, y, plot_title):
    xx, yy = np.meshgrid(np.linspace(-3, 3, 50), np.linspace(-3, 3, 50))
    clf.fit(X, y)
    # plot the decision function for each datapoint on the grid
    Z = clf.predict_proba(np.vstack((xx.ravel(), yy.ravel())).T)[:, 1]
    Z = Z.reshape(xx.shape)

    image = plt.imshow(
        Z,
        interpolation="nearest",
        extent=(xx.min(), xx.max(), yy.min(), yy.max()),
        aspect="auto",
        origin="lower",
        cmap=plt.cm.PuOr_r,
    )
    contours = plt.contour(xx, yy, Z, levels=[0], linewidths=2, linetypes="--")
    plt.scatter(X[:, 0], X[:, 1], s=30, c=y, cmap=plt.cm.Paired)
    plt.xticks(())
    plt.yticks(())
    plt.xlabel(r"$x_1$")
    plt.ylabel(r"$x_2$")
    plt.axis([-3, 3, -3, 3])
    plt.colorbar(image)
    plt.title(plot_title, fontsize=12);

In [ ]:
plot_boundary(
    LogisticRegression(solver="lbfgs"), X, y, "Logistic Regression, XOR problem"
)

Mais si l’on devait entrer des caractéristiques polynomiales (ici, jusqu’à 2 degrés), le problème est résolu.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
logit_pipe = Pipeline(
    [
        ("poly", PolynomialFeatures(degree=2)),
        ("logit", LogisticRegression(solver="lbfgs")),
    ]
)

In [ ]:
plot_boundary(logit_pipe, X, y, "Logistic Regression + quadratic features. XOR problem")

Ici, la régression logistique a toujours produit un hyperplan mais dans un espace à 6 dimensions $1, x_1, x_2, x_1^2, x_1x_2$ et $x_2^2$. Lorsque nous projetons vers l'espace de caractéristiques d'origine, $x_1, x_2$, la limite est non linéaire.

En pratique, les caractéristiques polynomiales aident, mais il est inefficace en calcul de les construire explicitement. SVM à noyau fonctionne beaucoup plus rapidement. Dans cette approche, seule la distance entre les objets (définie par la fonction du noyau) dans un espace de grande dimension est calculée et il n'est pas nécessaire de produire un nombre combinatoire important de caractéristiques.

### Ressources utiles
- Main course [site](https://mlcourse.ai), [course repo](https://github.com/Yorko/mlcourse.ai), and YouTube [channel](https://www.youtube.com/watch?v=QKTuw4PNOsU&list=PLVlY_7IJCMJeRfZ68eVfEcu-UcN9BbwiX)
- Medium ["story"](https://medium.com/open-machine-learning-course/open-machine-learning-course-topic-4-linear-classification-and-regression-44a41b9b5220) based on this notebook
- Course materials as a [Kaggle Dataset](https://www.kaggle.com/kashnitsky/mlcourse)
- If you read Russian: an [article](https://habrahabr.ru/company/ods/blog/323890/) on Habrahabr with ~ the same material. And a [lecture](https://youtu.be/oTXGQ-_oqvI) on YouTube
- A nice and concise overview of linear models is given in the book [“Deep Learning”](http://www.deeplearningbook.org) (I. Goodfellow, Y. Bengio, and A. Courville).
- Linear models are covered practically in every ML book. We recommend “Pattern Recognition and Machine Learning” (C. Bishop) and “Machine Learning: A Probabilistic Perspective” (K. Murphy).
- If you prefer a thorough overview of linear model from a statistician’s viewpoint, then look at “The elements of statistical learning” (T. Hastie, R. Tibshirani, and J. Friedman).
- The book “Machine Learning in Action” (P. Harrington) will walk you through implementations of classic ML algorithms in pure Python.
- [Scikit-learn](http://scikit-learn.org/stable/documentation.html) library. These guys work hard on writing really clear documentation.
- Scipy 2017 [scikit-learn tutorial](https://github.com/amueller/scipy-2017-sklearn) by Alex Gramfort and Andreas Mueller.
- One more [ML course](https://github.com/diefimov/MTH594_MachineLearning) with very good materials.
- [Implementations](https://github.com/rushter/MLAlgorithms) of many ML algorithms. Search for linear regression and logistic regression.